# Lives Saved Estimates with Python MINIMOD

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import pandas as pd
import minimod
from glob import glob
from pathlib import Path
import re
from functools import reduce
import geopandas as gpd
import matplotlib.pyplot as plt
import os
from ipywidgets import fixed, interact, IntSlider


In [3]:
# Updated Costs and Lives Saved

update_folder = Path('optimization_work', 'demewoz_lives_saved','data_files', 'updated_costs_lives_saved')

In [4]:
os.chdir("/home/lordflaron/Documents/minimod/")

## Processing Data

In [5]:
df_benefits_low = (
pd.read_excel(Path(update_folder, 'lives_saved_costs.xlsx'), 
sheet_name='lives_saved', 
skiprows=2)
.set_index(['intervention', 'region'])
.stack()
.reset_index()
.rename({0 : 'lives_saved', 'level_2' : 'time'}, axis=1)
)

df_benefits_high = (
pd.read_excel(Path(update_folder, 'lives_saved_costs.xlsx'), 
sheet_name='lives_saved_high', 
skiprows=2)
.set_index(['intervention', 'region'])
.stack()
.reset_index()
.rename({0 : 'lives_saved_high', 'level_2' : 'time'}, axis=1)
)

costs = (
pd.read_excel(Path(update_folder, 'lives_saved_costs.xlsx'), 
sheet_name='costs', 
skiprows=2)
.set_index(['intervention', 'region'])
.stack()
.reset_index()
.rename({0 : 'costs', 'level_2' : 'time'}, axis=1)
)

# Nope, load in Justin's data from GAMS
df_benefits_gams = (
pd.read_csv(Path(update_folder, 'lives_saved_low.csv'))
)

df_costs_gams = (
pd.read_csv(Path(update_folder, 'costs_low.csv'))
)


# Create function to make data adjustments
def observation_adjustment(data, int1, int2, time_to_replace, space_to_replace = slice(None)):
    
    df = data.copy(deep = True)

    if isinstance(int2, str):
        df_int2 = df.loc[(int2, space_to_replace, time_to_replace)]
        df.loc[(int1, space_to_replace, time_to_replace), :] = df_int2.values
    elif int2 == 0:
        df.loc[(int1, space_to_replace, time_to_replace), :] = 0

    print(f"Changed {int1} to {int2}")

    return df


In [6]:
# Create string function for going from GAMS code to python function

gams_str = """cov("cube",j,t2)=0                              ;
cov("cubezcube",j,t2)=0                         ;
cov("maxoilcube",j,t2)=cov("maxoil",j,t2) ;
cov("oilcube",j,t2)=cov("oil",j,t2)       ;
cov("oilcubevas",j,t2)=cov("oilvas",j,t2) ;
cov("maxoilcubevas",j,t2)=cov("maxoilvas",j,t2) ;
cov("cubevas",j,t2)=cov("vas",j,t2) ;
cov("cubeclinic",j,t2)=cov("clinic",j,t2) ;
cov("maxoilcubeclinic",j,t2)=cov("maxoilclinic",j,t2) ;
cov("oilcubeclinic",j,t2)=cov("oilclinic",j,t2) ;
cov("cubezflour",j,t2)=cov("zflour",j,t2) ;
cov("maxoilcubezflour",j,t2)=cov("maxoilzflour",j,t2) ;
cov("oilcubezflour",j,t2)=cov("oilzflour",j,t2) ;
cov("oilcubevaszflour",j,t2)=cov("oilvaszflour",j,t2) ;
cov("cubevaszflour",j,t2)=cov("vaszflour",j,t2) ;
cov("maxoilcubevaszflour",j,t2)=cov("maxoilvaszflour",j,t2) ;
cov("oilcubecliniczflour",j,t2)=cov("oilcliniczflour",j,t2) ;
cov("maxoilcubecliniczflour",j,t2)=cov("maxoilcliniczflour",j,t2) ;
cov("cubezflourzcube",j,t2)=cov("zflour",j,t2) ;
cov("maxoilcubezflourzcube",j,t2)=cov("maxoilzflour",j,t2) ;
cov("oilcubezflourzcube",j,t2)=cov("oilzflour",j,t2) ;
cov("oilcubevaszflourzcube",j,t2)=cov("oilvaszflour",j,t2) ;
cov("oilcubecliniczflourzcube",j,t2)=cov("oilcliniczflour",j,t2) ;
cov("maxoilcubecliniczflourzcube",j,t2)=cov("maxoilcliniczflour",j,t2) ;

cov("zcube",j,t2)=0                      ;
cov("zflourzcube",j,t2)=cov("zflour",j,t2) ;
cov("oilvaszflourzcube",j,t2)=cov("oilvaszflour",j,t2) ;
cov("maxoilcubezflourzcube",j,t2)=cov("maxoilzflour",j,t2) ;
cov("oilzflourzcube",j,t2)=cov("oilzflour",j,t2) ;

cov("oilzflourfcubefflour",j,t2)=cov("oilzflourfflour",j,t2) ;
cov("maxoil",j,t4)=cov("oil",j,t4) ;
cov("maxoilcube",j,t4)=cov("oilcube",j,t4) ;
cov("maxoilvas",j,t4)=cov("oilvas",j,t4) ;
cov("maxoilcubevas",j,t4)=cov("oilcubevas",j,t4) ;
cov("maxoilclinic",j,t4)=cov("oilclinic",j,t4) ;
cov("maxoilcubeclinic",j,t4)=cov("oilcubeclinic",j,t4) ;
cov("maxoilzflour",j,t4)=cov("oilzflour",j,t4) ;
cov("maxoilcubezflour",j,t4)=cov("oilcubezflour",j,t4) ;
cov("maxoilvaszflour",j,t4)=cov("oilvaszflour",j,t4) ;
cov("maxoilcliniczflour",j,t4)=cov("oilcliniczflour",j,t4) ;
cov("maxoilcubevaszflour",j,t4)=cov("oilcubevaszflour",j,t4) ;
cov("maxoilcubecliniczflour",j,t4)=cov("oilcubecliniczflour",j,t4) ;
cov("maxoilzflourzcube",j,t4)=cov("oilzflourzcube",j,t4) ;
cov("maxoilcubezflourzcube",j,t4)=cov("oilcubezflourzcube",j,t4) ;
cov("maxoilvaszflourzcube",j,t4)=cov("oilvaszflourzcube",j,t4) ;
cov("maxoilcliniczflourzcube",j,t4)=cov("oilcliniczflourzcube",j,t4) ;
cov("maxoilcubevaszflourzcube",j,t4)=cov("oilcubevaszflourzcube",j,t4) ;
cov("maxoilcubecliniczflourzcube",j,t4)=cov("oilcubecliniczflourzcube",j,t4) ;
cov("zflourfflour33",j,t)=0;
cov("fflour33",j,t)=0;"""

In [7]:
a = [x.strip() for x in gams_str.split(';')]

for i in a:

    cov_str = re.compile(r'(?<=cov\()(\"[a-z0-9]+\")')
    time_str = re.compile(r't[24]*')

    found = cov_str.findall(i)
    time_found = time_str.findall(i)
    try:
        # print(time_found)
        if time_found[0] == 't2':
            time_range = '[1,2,3]'
        elif time_found[0] == 't4':
            time_range = '[1,2]'
        elif time_found[0] == 't':
            time_range = 'slice(None)'
        elif not time_found:
            print("not found time")
            continue

        if len(found) == 2:
            second_int = found[1]
        elif not found:
            print("not found")
        else:
            second_int = 0

        # print(found, time_range)
        print(f""".pipe(observation_adjustment,\nint1 = {found[0]},\nint2 = {second_int},\ntime_to_replace = {time_range})""")
        
    except IndexError:
        continue



.pipe(observation_adjustment,
int1 = "cube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezcube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcube",
int2 = "maxoil",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcube",
int2 = "oil",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubevas",
int2 = "oilvas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubevas",
int2 = "maxoilvas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubevas",
int2 = "vas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubeclinic",
int2 = "clinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubeclinic",
int2 = "maxoilclinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubeclinic",
int2 = "oilclinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezflour",
int2 = "

In [8]:
df_benefits_low_adjusted = (df_benefits_low.set_index(['intervention', 'region', 'time'])
.pipe(observation_adjustment,
int1 = "cube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezcube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcube",
int2 = "maxoil",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcube",
int2 = "oil",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubevas",
int2 = "oilvas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubevas",
int2 = "maxoilvas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubevas",
int2 = "vas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubeclinic",
int2 = "clinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubeclinic",
int2 = "maxoilclinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubeclinic",
int2 = "oilclinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezflour",
int2 = "zflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubezflour",
int2 = "maxoilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubezflour",
int2 = "oilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubevaszflour",
int2 = "oilvaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubevaszflour",
int2 = "vaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubevaszflour",
int2 = "maxoilvaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubecliniczflour",
int2 = "oilcliniczflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubecliniczflour",
int2 = "maxoilcliniczflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezflourzcube",
int2 = "zflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubezflourzcube",
int2 = "maxoilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubezflourzcube",
int2 = "oilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubevaszflourzcube",
int2 = "oilvaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubecliniczflourzcube",
int2 = "oilcliniczflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubecliniczflourzcube",
int2 = "maxoilcliniczflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "zcube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "zflourzcube",
int2 = "zflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilvaszflourzcube",
int2 = "oilvaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubezflourzcube",
int2 = "maxoilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilzflourzcube",
int2 = "oilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilzflourfcubefflour",
int2 = "oilzflourfflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoil",
int2 = "oil",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcube",
int2 = "oilcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilvas",
int2 = "oilvas",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubevas",
int2 = "oilcubevas",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilclinic",
int2 = "oilclinic",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubeclinic",
int2 = "oilcubeclinic",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilzflour",
int2 = "oilzflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubezflour",
int2 = "oilcubezflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilvaszflour",
int2 = "oilvaszflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcliniczflour",
int2 = "oilcliniczflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubevaszflour",
int2 = "oilcubevaszflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubecliniczflour",
int2 = "oilcubecliniczflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilzflourzcube",
int2 = "oilzflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubezflourzcube",
int2 = "oilcubezflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilvaszflourzcube",
int2 = "oilvaszflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcliniczflourzcube",
int2 = "oilcliniczflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubevaszflourzcube",
int2 = "oilcubevaszflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubecliniczflourzcube",
int2 = "oilcubecliniczflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "zflourfflour33",
int2 = 0,
time_to_replace = slice(None))
.pipe(observation_adjustment,
int1 = "fflour33",
int2 = 0,
time_to_replace = slice(None)))


Changed cube to 0
Changed cubezcube to 0
Changed maxoilcube to maxoil
Changed oilcube to oil
Changed oilcubevas to oilvas
Changed maxoilcubevas to maxoilvas
Changed cubevas to vas
Changed cubeclinic to clinic
Changed maxoilcubeclinic to maxoilclinic
Changed oilcubeclinic to oilclinic
Changed cubezflour to zflour
Changed maxoilcubezflour to maxoilzflour
Changed oilcubezflour to oilzflour
Changed oilcubevaszflour to oilvaszflour
Changed cubevaszflour to vaszflour
Changed maxoilcubevaszflour to maxoilvaszflour
Changed oilcubecliniczflour to oilcliniczflour
Changed maxoilcubecliniczflour to maxoilcliniczflour
Changed cubezflourzcube to zflour
Changed maxoilcubezflourzcube to maxoilzflour
Changed oilcubezflourzcube to oilzflour
Changed oilcubevaszflourzcube to oilvaszflour
Changed oilcubecliniczflourzcube to oilcliniczflour
Changed maxoilcubecliniczflourzcube to maxoilcliniczflour
Changed zcube to 0
Changed zflourzcube to zflour
Changed oilvaszflourzcube to oilvaszflour
Changed maxoilcubezf

In [9]:
df_benefits_high_adjusted = (df_benefits_high.set_index(['intervention', 'region', 'time'])
.pipe(observation_adjustment,
int1 = "cube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezcube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcube",
int2 = "maxoil",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcube",
int2 = "oil",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubevas",
int2 = "oilvas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubevas",
int2 = "maxoilvas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubevas",
int2 = "vas",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubeclinic",
int2 = "clinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubeclinic",
int2 = "maxoilclinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubeclinic",
int2 = "oilclinic",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezflour",
int2 = "zflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubezflour",
int2 = "maxoilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubezflour",
int2 = "oilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubevaszflour",
int2 = "oilvaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubevaszflour",
int2 = "vaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubevaszflour",
int2 = "maxoilvaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubecliniczflour",
int2 = "oilcliniczflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubecliniczflour",
int2 = "maxoilcliniczflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezflourzcube",
int2 = "zflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubezflourzcube",
int2 = "maxoilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubezflourzcube",
int2 = "oilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubevaszflourzcube",
int2 = "oilvaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubecliniczflourzcube",
int2 = "oilcliniczflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubecliniczflourzcube",
int2 = "maxoilcliniczflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "zcube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "zflourzcube",
int2 = "zflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilvaszflourzcube",
int2 = "oilvaszflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcubezflourzcube",
int2 = "maxoilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilzflourzcube",
int2 = "oilzflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilzflourfcubefflour",
int2 = "oilzflourfflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoil",
int2 = "oil",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcube",
int2 = "oilcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilvas",
int2 = "oilvas",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubevas",
int2 = "oilcubevas",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilclinic",
int2 = "oilclinic",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubeclinic",
int2 = "oilcubeclinic",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilzflour",
int2 = "oilzflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubezflour",
int2 = "oilcubezflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilvaszflour",
int2 = "oilvaszflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcliniczflour",
int2 = "oilcliniczflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubevaszflour",
int2 = "oilcubevaszflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubecliniczflour",
int2 = "oilcubecliniczflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilzflourzcube",
int2 = "oilzflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubezflourzcube",
int2 = "oilcubezflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilvaszflourzcube",
int2 = "oilvaszflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcliniczflourzcube",
int2 = "oilcliniczflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubevaszflourzcube",
int2 = "oilcubevaszflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcubecliniczflourzcube",
int2 = "oilcubecliniczflourzcube",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "zflourfflour33",
int2 = 0,
time_to_replace = slice(None))
.pipe(observation_adjustment,
int1 = "fflour33",
int2 = 0,
time_to_replace = slice(None))
.pipe(observation_adjustment,
int1 = "cubezflourzcubefcubefflour",
int2 = "zflourfflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "cubezflourfcubefflour",
int2 = "zflourfflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilzflourzcubefcubefflour",
int2 = "oilzflourfflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "fcube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "fcube",
int2 = 0,
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilfcube",
int2 = 'oil',
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "oilcubeclinicfcube",
int2 = 'oilclinic',
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilzflourfcubefflour",
int2 = 'maxoilzflourfflour',
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilcliniczflourfcubefflour",
int2 = "maxoilcliniczflourfflour",
time_to_replace = [1,2,3])
.pipe(observation_adjustment,
int1 = "maxoilzflourfcubefflour",
int2 = "oilzflourfcubefflour",
time_to_replace = [1,2])
.pipe(observation_adjustment,
int1 = "maxoilcliniczflourfcubefflour",
int2 = "oilcliniczflourfcubefflour",
time_to_replace = [1,2])
)


Changed cube to 0
Changed cubezcube to 0
Changed maxoilcube to maxoil
Changed oilcube to oil
Changed oilcubevas to oilvas
Changed maxoilcubevas to maxoilvas
Changed cubevas to vas
Changed cubeclinic to clinic
Changed maxoilcubeclinic to maxoilclinic
Changed oilcubeclinic to oilclinic
Changed cubezflour to zflour
Changed maxoilcubezflour to maxoilzflour
Changed oilcubezflour to oilzflour
Changed oilcubevaszflour to oilvaszflour
Changed cubevaszflour to vaszflour
Changed maxoilcubevaszflour to maxoilvaszflour
Changed oilcubecliniczflour to oilcliniczflour
Changed maxoilcubecliniczflour to maxoilcliniczflour
Changed cubezflourzcube to zflour
Changed maxoilcubezflourzcube to maxoilzflour
Changed oilcubezflourzcube to oilzflour
Changed oilcubevaszflourzcube to oilvaszflour
Changed oilcubecliniczflourzcube to oilcliniczflour
Changed maxoilcubecliniczflourzcube to maxoilcliniczflour
Changed zcube to 0
Changed zflourzcube to zflour
Changed oilvaszflourzcube to oilvaszflour
Changed maxoilcubezf

In [10]:
# Merge data
df = (
df_benefits_low_adjusted
.merge(df_benefits_high_adjusted, on = ['intervention', 'region', 'time'])
.merge(costs, on = ['intervention', 'region', 'time'])
.assign(lives_saved = lambda df: df['lives_saved'].astype(int),
lives_saved_high = lambda df: df['lives_saved_high'].astype(int),
costs = lambda df: df['costs'].astype(int))
)

df_gams = (
df_benefits_gams
.merge(df_costs_gams, on = ['intervention', 'space', 'time'])
)




## Summary Statistics

Now we show some summary statistics of the data

### Lives Saved

In [15]:
s = minimod.utils.summary.PreOptimizationDataSummary(
    data = df,
    benefit_col= 'lives_saved',
    cost_col= 'costs',
    intervention_col='intervention',
    space_col= 'region',
    time_col='time',
    benefit_title='Lives Saved (Low)',
    intervention_subset=['clinic', 'vas','zflour', 'fflour', 'cube', 'fcube','oil', 'maxoil'],
    intervention_subset_titles={'clinic' : 'VAS Clinic Day', 
    'vas' : 'Vitamin A Supp.', 
    'zflour' : 'Zinc Flour',
    'fflour' : 'Folic Acid Flour' ,
    'cube' : 'Boullion Cube',
    'fcube' : 'Boullion Cube with Folic Acid',
    'oil' : 'VA Oil 9 mg/kg',
    'maxoil' : "VA Oil 12 mg/kg",
    'oilvaszflourfflour33' : 'BAU: Oil/VAS/Flour with Zinc and Folic Acid (33%)'},
    bau_intervention = 'oilvaszflourfflour33'
)

s.summary_table(variables_of_interest= {'Cost per Child ($)' : 'cost_per_benefit'}, grouping = 'over_space', style = 'markdown')


,Intervention,Regions,Lives Saved (Low),Costs ($),Cost per Child ($)
2,VAS Clinic Day,North,"1,752","2,865,441",1635.530000
3,VAS Clinic Day,South,855,"3,841,014",4492.410000
0,VAS Clinic Day,Cities,413,"1,122,789",2718.620000
1,VAS Clinic Day,National,"3,020","7,829,244",2592.460000
30,Vitamin A Supp.,North,"10,886","10,255,671",942.100000
31,Vitamin A Supp.,South,"5,756","13,732,662",2385.800000
28,Vitamin A Supp.,Cities,"2,819","5,679,743",2014.810000
29,Vitamin A Supp.,National,"19,461","29,668,076",1524.490000
34,Zinc Flour,North,"3,365","1,433,316",425.950000
35,Zinc Flour,South,"3,353","2,266,593",675.990000


### Lives Saved High

In [27]:
s = minimod.utils.summary.PreOptimizationDataSummary(
    data = df,
    benefit_col= 'lives_saved_high',
    cost_col= 'costs',
    intervention_col='intervention',
    space_col= 'region',
    time_col='time',
    benefit_title='Lives Saved (High)',
    intervention_subset=['vas','zflour', 'fflour', 'fcube', 'oil', 'maxoil'],
    intervention_subset_titles={
    'vas' : 'Vitamin A Supp.', 
    'zflour' : 'Zinc Flour',
    'fflour' : 'Folic Acid Flour' ,
    'fcube' : 'Boullion Cube with Folic Acid',
    'oil' : 'VA Oil 9mg/kg',
    'maxoil' : 'VA Oil 12 mg/kg',
    'oilvaszflourfflour33' : 'BAU: Oil/VAS/Flour with Zinc and Folic Acid (33%)'},
    bau_intervention = 'oilvaszflourfflour33'
)

with open('table.html', 'w') as f:
    f.write(s.summary_table(variables_of_interest= {'Cost per Child ($)' : 'cost_per_benefit'}, grouping = 'over_space', style = 'markdown').render())

## Run MINIMOD

We're going to run models for every type of constraint and lives_saved and lives_saved_high with base, time, space and both constraints

In [16]:
models = {}

for col in ['lives_saved', 'lives_saved_high']:

    for opt_type in [None]: #'time', 'space', 'both']:

        print(f"Running {col} with {opt_type}")

        m= minimod.Minimod(solver_type = 'costmin',
        data = df,
        benefit_col = col,
        cost_col = 'costs',
        space_col = 'region',
        all_space = ['(?=.*(?<![zf])cube)', 'zcube', 'fcube', '(?=.*(?<![max])oil)', 'maxoil', 'zflour', 'fflour(?!33)', 'fflour33'],
        all_time = ['(?=.*(?<![max])oil)', 'maxoil','zflour', 'fflour','fflour(?!33)', '(?=.*(?<![zf])cube)' , 'zcube', 'fcube'],
        time_subset = [1,2,3],
        benefit_title = 'Lives Saved',
        minimum_benefit = 'oilvaszflourfflour33',
        main_constraint_over = opt_type
        )

        models.setdefault(col, {})[opt_type] = m.fit()

Running lives_saved with None
[Note]: Processing Data...
[Note]: Creating Base Model with constraints

                MiniMod Nutrition Intervention Tool
                Optimization Method: MIN
                Version: 0.0.6dev
                Solver: CBC,
                Show Output: True
                
                
[Note]: Optimizing...
[Note]: Optimal Solution Found
Running lives_saved_high with None
[Note]: Processing Data...
[Note]: Creating Base Model with constraints

                MiniMod Nutrition Intervention Tool
                Optimization Method: MIN
                Version: 0.0.6dev
                Solver: CBC,
                Show Output: True
                
                
[Note]: Optimizing...
[Note]: Optimal Solution Found


### Results of the Model Optimization

In [17]:
# Load data
geo_df = gpd.read_file("examples/data/maps/cameroon/CAM.shp")

# Now we create the boundaries for North, South and Cities
# Based on "Measuring Costs of Vitamin A..., Table 2"
north = r"Adamaoua|Nord|Extreme-Nord"
south = r"Centre|Est|Nord-Ouest|Ouest|Sud|Sud-Ouest"
cities= r"Littoral" # Duala
# Yaounde is in Mfoundi
geo_df.loc[lambda df: df['ADM1'].str.contains(north), 'space'] = 'North'
geo_df.loc[lambda df: df['ADM1'].str.contains(south), 'space'] = 'South'
geo_df.loc[lambda df: df['ADM1'].str.contains(cities), 'space'] = 'Cities'
geo_df.loc[lambda df: df['ADM2'].str.contains(r"Mfoundi"), 'space'] = 'Cities'

# Now we aggregate the data to the `space` variable
agg_geo_df = geo_df.dissolve(by = 'space')


#### Reports

In [18]:
def report_model(col, opt_type):

    m = models[col][opt_type]

    return getattr(m, 'report')()

In [19]:
interact(report_model, 
col = ['lives_saved', 'lives_saved_high'],
opt_type = [None, 'time', 'space', 'both'])

interactive(children=(Dropdown(description='col', options=('lives_saved', 'lives_saved_high'), value='lives_sa…

<function __main__.report_model(col, opt_type)>

#### BAU Time Plots

In [17]:
def plot_bau_model(col, opt_type):

    m = models[col][opt_type]

    fig, (ax1, ax2) = plt.subplots(1,2,  figsize = (8,5))
    getattr(m, 'plot_bau_time')(ax=ax1)
    getattr(m, 'plot_bau_time')(opt_variable = 'c', ax=ax2)


In [18]:
interact(plot_bau_model, 
col = ['lives_saved', 'lives_saved_high'],
opt_type = [None, 'time', 'space', 'both'])

interactive(children=(Dropdown(description='col', options=('lives_saved', 'lives_saved_high'), value='lives_sa…

<function __main__.plot_bau_model(col, opt_type)>

#### Maps

In [19]:
def plot_map_model(col, opt_type, **kwargs):
    
    m = models[col][opt_type]

    return getattr(m, 'plot_map_benchmark')(**kwargs)


In [22]:
interact(plot_map_model,
col = ['lives_saved', 'lives_saved_high'],
opt_type = [None, 'time', 'space', 'both'],
intervention = fixed(None), 
time = IntSlider(min=1, max=10, step=1, value=1), 
optimum_interest = ['b', 'c', 'cdb', 'cdc', 'cb', 'cc'], 
bench_intervention = fixed('oilvaszflourfflour33'),
map_df = fixed(agg_geo_df),
merge_key = fixed('space'),
intervention_in_title = fixed(True),
intervention_bubbles= fixed(True),
intervention_bubble_names = fixed(['oil', 'vas', 'clinic', 'fcube', 'zflour', 'fflour', 'fflour33']),
bau_intervention_bubble_names = fixed(['oil', 'vas', 'clinic', 'fcube', 'zflour', 'fflour33']),
save = fixed(None))

interactive(children=(Dropdown(description='col', options=('lives_saved', 'lives_saved_high'), value='lives_sa…

<function __main__.plot_map_model(col, opt_type, **kwargs)>

In [21]:
models['lives_saved'][None].bau_list

region  time
Cities  1       oilvaszflourfflour33
        2       oilvaszflourfflour33
        3       oilvaszflourfflour33
        4       oilvaszflourfflour33
        5       oilvaszflourfflour33
        6       oilvaszflourfflour33
        7       oilvaszflourfflour33
        8       oilvaszflourfflour33
        9       oilvaszflourfflour33
        10      oilvaszflourfflour33
North   1       oilvaszflourfflour33
        2       oilvaszflourfflour33
        3       oilvaszflourfflour33
        4       oilvaszflourfflour33
        5       oilvaszflourfflour33
        6       oilvaszflourfflour33
        7       oilvaszflourfflour33
        8       oilvaszflourfflour33
        9       oilvaszflourfflour33
        10      oilvaszflourfflour33
South   1       oilvaszflourfflour33
        2       oilvaszflourfflour33
        3       oilvaszflourfflour33
        4       oilvaszflourfflour33
        5       oilvaszflourfflour33
        6       oilvaszflourfflour33
        7       oilvaszfl